## Imports ##

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tqdm import tqdm
import sgf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.utils import to_categorical



## Fonctions Utilitaires ##

In [ ]:
def sgf_coords_to_indices(coord, board_size):
    """Convert SGF coordinates (e.g., 'pd') to array indices."""
    col, row = ord(coord[0]) - ord('a'), ord(coord[1]) - ord('a')
    return board_size - row - 1, col
def sgf_to_sequence(sgf_file, board_size=19):
    """
    Convert an SGF file to a sequence of Go board states.
    
    Args:
        sgf_file (str): Path to the SGF file.
        board_size (int): Size of the Go board.
    
    Returns:
        sequence (list of np.array): Sequence of board states.
    """
    with open(sgf_file, 'r') as f:
        sgf_content = f.read()    
    collection = sgf.parse(sgf_content)
    game = collection[0]  # Assume a single game
    board = np.zeros((board_size, board_size), dtype=int)
    sequence = [board.copy()]
    
    for node in game.rest:
        move = node.properties
        if 'B' in move:  # Black move
            x, y = sgf_coords_to_indices(move['B'][0], board_size)
            board[x, y] = 1
        elif 'W' in move:  # White move
            x, y = sgf_coords_to_indices(move['W'][0], board_size)
            board[x, y] = 2
        sequence.append(board.copy())
    
    return sequence
def load_games_from_folder(folder_path, board_size=19):
    """
    Load all SGF files from a folder and convert them to sequences of Go board states.
    
    Args:
        folder_path (str): Path to the folder containing SGF files.
        board_size (int): Size of the Go board.
    
    Returns:
        games (list of list of np.array): List of games, each represented as a sequence of board states.
    """
    games = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.sgf'):
            sgf_file_path = os.path.join(folder_path, filename)
            game_sequence = sgf_to_sequence(sgf_file_path, board_size)
            games.append(game_sequence)
    return games

## Préparation des données ##

In [ ]:
folder_path = r"training set"
games = load_games_from_folder(folder_path)

In [ ]:
def create_training_data(games):
    X, y_moves = [], []
    for game in games:
        for i in range(len(game) - 1):
            board_state = game[i]
            next_board_state = game[i + 1]
            
            # Find the move that changed the board
            move = np.argwhere(next_board_state != board_state)
            if len(move) == 1:  # Ensure one valid move
                x, y = move[0]
                X.append(board_state)
                y_moves.append((x, y))
    
    X = np.array(X)
    y_moves = np.array(y_moves)
    return X, y_moves

X, y_moves = create_training_data(games)

# Convert moves to categorical indices
unique_moves = {move: idx for idx, move in enumerate(set(map(tuple, y_moves)))}
move_to_coords = {idx: move for move, idx in unique_moves.items()}  # Reverse mapping
y = np.array([unique_moves[tuple(move)] for move in y_moves])

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Architectrue du modèle ##

In [ ]:

model = Sequential([
    Input(shape=X_train.shape[1:]),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(unique_moves), activation='softmax')  # Output layer for move probabilities
])

model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3000, batch_size=256)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_4 (Flatten)             │ (None, 361)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        46,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 361)            │        23,465 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,057 (304.91 KB)

 Trainable params: 78,057 (304.91 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3000
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0030 - loss: 5.9010 - val_accuracy: 0.0029 - val_loss: 5.8820
Epoch 2/3000
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0045 - loss: 5.8580 - val_accuracy: 0.0062 - val_loss: 5.8186
Epoch 3/3000
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0086 - loss: 5.7441 - val_accuracy: 0.0080 - val_loss: 5.7277
Epoch 4/3000
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0126 - loss: 5.6097 - val_accuracy: 0.0087 - val_loss: 5.6304
Epoch 5/3000
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0159 - loss: 5.4544 - val_accuracy: 0.0097 - val_loss: 5.5384
Epoch 6/3000
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0202 - loss: 5.3029 - val_accuracy: 0.0112 - val_loss: 5.4661
Epoch 7/3000
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0252 - loss: 5.1796 - val_accuracy: 0.0114 - val_loss: 5.4182
Epoch 8/3000
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0294 - loss: 5.0630 - 

Sauvegarde du modèle entrainé pour utilisation ultérieure

In [ ]:
model.save('modelCNN.keras')